In [1]:
import pandas as pd
full_df = pd.read_csv('train-v7.csv')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances

import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
def cleaning_text(sentence):
    sentence = str(sentence)
    sentence=sentence.lower()
    sentence=re.sub('[^\w\s]',' ', sentence) #removes punctuations
    sentence=re.sub('_',' ', sentence) #removes punctuations
    sentence=re.sub('\d+',' ', sentence) #removes digits
    cleaned=' '.join([w for w in sentence.split() if not w in stop]) # removes english stopwords
    cleaned=' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])
    
#selecting only nouns and adjectives
    cleaned=' '.join([w for w in cleaned.split() if not len(w)<=2 ]) #removes single lettered words and digits
    cleaned=cleaned.strip()
    return cleaned
	  
full_df['cleaned'] = full_df['description'].apply(lambda x: cleaning_text(x))

In [2]:
trimDF_train = full_df[['aptID','cleaned','interest_level']]
trimDF_train.to_csv('trimDF_train_v7_200.csv')

In [8]:
from gensim.models import doc2vec
import nltk
import re

def split_sentence(sentence):
    words = re.split('\W+', sentence.lower())
    return [word for word in words if word != ""]

class MyDocs(object):
    def __iter__(self):
        for i, text in enumerate(open("trimDF_train_v7_200.csv")):
            if i:
                yield doc2vec.LabeledSentence(words=split_sentence(text), tags=['%s' % i])

# Train the doc2vec model
mydocs = MyDocs()
model = doc2vec.Doc2Vec(mydocs, size = 200, window = 8, min_count = 5, workers = 4)
model.save('description.model')

In [9]:
test = pd.read_csv('trimDF_train_v7_200.csv')
test['docvecs'] = model.docvecs

In [12]:
test['docvecs'].apply(pd.Series).add_prefix('value')

,value0,value1,value2,value3,value4,value5,value6,value7,value8,value9,...,value190,value191,value192,value193,value194,value195,value196,value197,value198,value199
0,-0.059732,0.057407,-0.018174,0.029979,0.028831,-0.042933,0.005786,-0.071537,0.051735,-0.093637,...,-0.003807,-0.058529,-0.032566,-0.005951,0.079763,0.045382,-0.009145,-0.063899,-0.028969,0.035853
1,-0.106078,0.033969,-0.012465,0.050670,0.078883,-0.036583,0.045765,-0.130474,0.099986,-0.107313,...,-0.004898,-0.128851,-0.020444,-0.082270,0.105996,0.128762,0.043672,-0.054171,-0.003532,0.011584
2,-0.042874,0.011087,-0.021669,0.027445,0.061559,-0.038549,-0.046897,-0.045344,0.054926,-0.037617,...,-0.017200,-0.068239,0.014053,-0.044400,0.061849,0.056064,-0.011448,-0.055422,0.009567,0.049901
3,-0.061724,0.008699,-0.021910,0.028907,0.030388,-0.024214,-0.006774,-0.019945,-0.011748,-0.055008,...,-0.036495,0.012032,-0.022381,0.018687,-0.002026,0.004172,-0.003661,-0.071646,0.020114,-0.006963
4,-0.028108,0.018300,0.034919,0.018705,0.021637,-0.021738,0.012277,-0.051925,0.063538,-0.046254,...,-0.007690,-0.031833,-0.004263,-0.027247,0.067958,0.018767,0.003605,-0.044860,-0.013817,0.010109
5,-0.004160,-0.036662,-0.004671,0.035219,0.012800,0.000752,0.021145,-0.054919,0.074128,-0.021303,...,0.004457,-0.035541,0.045486,0.008899,0.026614,-0.016036,0.040457,0.047487,-0.012281,0.003762
6,0.008564,0.001060,-0.010770,0.017290,0.041159,-0.037475,0.005028,-0.022412,0.052985,-0.039668,...,-0.028301,-0.009128,-0.063302,-0.002361,0.020031,0.017003,0.010839,-0.024270,0.009947,0.028034
7,-0.022899,0.051039,0.009751,0.044974,0.000320,-0.000949,-0.035007,-0.025257,0.030476,-0.071265,...,0.042569,-0.030097,-0.013914,-0.012029,0.083429,-0.005882,-0.038416,0.003577,0.008087,0.020136
8,-0.011097,0.008412,0.003260,0.032604,-0.002798,-0.003621,0.000526,-0.030667,0.025442,-0.017015,...,-0.031914,-0.021772,0.006803,-0.025749,-0.000596,-0.010038,0.025556,-0.017708,-0.005735,0.014137
9,-0.034924,0.030126,-0.040491,0.103263,0.080333,-0.083814,-0.031995,-0.097329,0.045988,-0.117642,...,-0.047326,-0.047266,-0.019499,0.034822,0.109067,0.045263,-0.020000,-0.110012,0.008977,0.053052


In [13]:
test = pd.concat([test.drop(['docvecs'], axis=1), test['docvecs'].apply(pd.Series).add_prefix('value')], axis=1)

In [15]:
test.head()

,Unnamed: 0,aptID,cleaned,interest_level,value0,value1,value2,value3,value4,value5,...,value190,value191,value192,value193,value194,value195,value196,value197,value198,value199
0,0,4,spacious bedroom bathroom williamsburg apartme...,medium,-0.059732,0.057407,-0.018174,0.029979,0.028831,-0.042933,...,-0.003807,-0.058529,-0.032566,-0.005951,0.079763,0.045382,-0.009145,-0.063899,-0.028969,0.035853
1,1,6,brand new gut true bedroomfind home center gra...,low,-0.106078,0.033969,-0.012465,0.050670,0.078883,-0.036583,...,-0.004898,-0.128851,-0.020444,-0.082270,0.105996,0.128762,0.043672,-0.054171,-0.003532,0.011584
2,2,9,flex bedroom full wall perfect apartment east ...,medium,-0.042874,0.011087,-0.021669,0.027445,0.061559,-0.038549,...,-0.017200,-0.068239,0.014053,-0.044400,0.061849,0.056064,-0.011448,-0.055422,0.009567,0.049901
3,3,10,brand new bedroom bath apartmentenjoy apartmen...,medium,-0.061724,0.008699,-0.021910,0.028907,0.030388,-0.024214,...,-0.036495,0.012032,-0.022381,0.018687,-0.002026,0.004172,-0.003661,-0.071646,0.020114,-0.006963
4,4,15,studio abundant available rent case case minim...,low,-0.028108,0.018300,0.034919,0.018705,0.021637,-0.021738,...,-0.007690,-0.031833,-0.004263,-0.027247,0.067958,0.018767,0.003605,-0.044860,-0.013817,0.010109


In [24]:
colsToDrop = ['Unnamed: 0','cleaned','interest_level']
description_vec = test.drop(colsToDrop, axis=1)

In [16]:
test.columns

Index([u'Unnamed: 0', u'aptID', u'cleaned', u'interest_level', u'value0',
       u'value1', u'value2', u'value3', u'value4', u'value5',
       ...
       u'value190', u'value191', u'value192', u'value193', u'value194',
       u'value195', u'value196', u'value197', u'value198', u'value199'],
      dtype='object', length=204)

In [25]:
description_vec.to_csv('description_vec_train_v7_200.csv', index =False)